In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.layers import BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

import kerastuner
from kerastuner.tuners import Hyperband, BayesianOptimization

import numpy as np
import pandas as pd

In [ ]:
data_path = 'data/TwinsUK.xls'

tw_train_data = pd.read_excel(data_path, sheet_name='Training Set')
tw_test_data = pd.read_excel(data_path, sheet_name='Testing Set')

twins_data = pd.concat([tw_train_data, tw_test_data], ignore_index = True)

In [ ]:
# Data & model configuration
batch_size = 32
no_epochs = 1000
latent_dim = 18

original_dim = tw_train_data.shape[1]
input_shape = (original_dim,)

In [ ]:
# recommended to do this here: https://www.tensorflow.org/tutorials/keras/keras_tuner
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)
        
        
def model_builder(hp):
    # # =================
    # # Encoder
    # # =================

    # Definition
    i       = Input(shape=input_shape, name='encoder_input')
    
    x       = Dense(hp.Int('encoder_units',
                           min_value=30,
                           max_value=220,
                           step=10))(i)
    x       = LeakyReLU()(x)
    
    mu      = Dense(latent_dim, name='latent_mu')(x)
    sigma   = Dense(latent_dim, name='latent_sigma')(x)

    # Define sampling with reparameterization trick
    def sample_z(args):
        mu, sigma = args
        batch     = K.shape(mu)[0]
        dim       = K.int_shape(mu)[1]
        eps       = K.random_normal(shape=(batch, dim))
        return mu + K.exp(sigma / 2) * eps

    # Use reparameterization trick to ....??
    z       = Lambda(sample_z, output_shape=(latent_dim, ), name='z')([mu, sigma])

    # Instantiate encoder
    encoder = Model(i, [mu, sigma, z], name='encoder')
    
    # =================
    # Decoder
    # =================

    # Definition
    d_i   = Input(shape=(latent_dim, ), name='decoder_input')
    
    x     = Dense(hp.Int('decoder_units',
                           min_value=20,
                           max_value=220,
                           step=10))(d_i)
    x     = LeakyReLU()(x)
        
    o     = Dense(original_dim)(x)

    # Instantiate decoder
    decoder = Model(d_i, o, name='decoder')
    
    # =================
    # VAE as a whole
    # =================

    # Define loss
    def kl_reconstruction_loss(true, pred):
      # Reconstruction loss
        reconstruction_loss = mse(true, pred)
        reconstruction_loss *= original_dim

        # KL divergence loss
        kl_loss = 1 + sigma - K.square(mu) - K.exp(sigma)
        kl_loss = K.sum(kl_loss, axis=-1)
        kl_loss *= -0.5
        
        # weight KL divergence loss here
        kl_loss *= hp.Float(
        'kl_beta',
        min_value=1e-3,
        max_value=1e1,
        sampling='LOG',
        default=1e-2
        )

        return K.mean(reconstruction_loss + kl_loss)

    # Instantiate VAE
    vae_outputs = decoder(encoder(i)[2])
    vae         = Model(i, vae_outputs, name='vae')


    # Define optimizer
    optimizer = Adam(hp.Float(
        'learning_rate',
        min_value=1e-4,
        max_value=1e-2,
        sampling='LOG',
        default=1e-3
    ))

    # Compile VAE
    vae.compile(optimizer=optimizer, loss=kl_reconstruction_loss, metrics = ['mse'], experimental_run_tf_function=False)
    
    return vae
    

In [ ]:
# Set tuner parameters
tuner = Hyperband(
    model_builder,
    objective='mse',
    factor=2,
    max_epochs=200,
    directory='hyperband_optimization',
    project_name='mtvae')


In [ ]:
tuner.search_space_summary()

In [ ]:
# Run Tuner
# Runtime on MacBook Pro 2017: 01h 15m
tuner.search(
    tw_train_data, tw_train_data, 
    validation_data = (tw_test_data, tw_test_data))

## Print out best tuned parameters

In [ ]:
tuner.results_summary(num_trials = 3)

In [ ]:
tuner.get_best_models()[0].summary()

In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(best_hps.get('encoder_units'))
print(best_hps.get('decoder_units'))
print(best_hps.get('learning_rate'))
print(best_hps.get('kl_beta'))